The purpose of this file is to load World bank indicator data, obtained in CSV format, downloading from the [World Bank Indicator portal](http://datatopics.worldbank.org/world-development-indicators/)  

The data is reformated to a pandas dataframe with following format specified: indicators are on columns and the country and year form a multi level row index

In [1]:
import pandas as pd
import json
from pandas_datareader import wb

In [3]:
data_dir = '.\\..\\..\\data\\'
#name of output pickle file
world_bank_file_out = "world_bank_bulk_data.pkl"
#Name of input bulk csv file from the World bank website
world_bank_file_input = "WDIData.csv"
wb_data = pd.read_csv(data_dir + world_bank_file_input)

#### Format as received from the World Bank:

In [11]:
print("Shape:",wb_data.shape)

wb_data.head(3)

Shape: (12361, 1594)


Indicator Code    EG.CFT.ACCS.ZS  EG.ELC.ACCS.ZS  ...  NY.GSR.NFCY.KN  \
Country     Year                                  ...                   
Afghanistan 1972             NaN             NaN  ...             NaN   
            1973             NaN             NaN  ...             NaN   
            1974             NaN             NaN  ...             NaN   

Indicator Code    SH.STA.FGMS.ZS  
Country     Year                  
Afghanistan 1972             NaN  
            1973             NaN  
            1974             NaN  

[3 rows x 1594 columns]

In [7]:
#filter out innecessary columns (we will focus on years 1972 to 2018)
drop = ['Country Code','Indicator Name','1960','1961','1962','1963','1964',\
        '1965','1966','1967','1968','1969', '1970', '1971', 'Unnamed: 63']
wb_data = wb_data.drop(drop, axis='columns')

In [8]:
#Switch around the data so that indicators are on columns and the country and year form a multi level row index
wb_data = wb_data.set_index(['Country Name', 'Indicator Code'])
wb_data = wb_data.stack()
wb_data = wb_data.unstack(['Indicator Code'])
wb_data = wb_data.sort_index()
wb_data.index.levels[1].name = 'Year'
wb_data.index.levels[0].name = 'Country'

#### Output format:

In [9]:
wb_data.head(10)

Indicator Code    EG.CFT.ACCS.ZS  EG.ELC.ACCS.ZS  ...  NY.GSR.NFCY.KN  \
Country     Year                                  ...                   
Afghanistan 1972             NaN             NaN  ...             NaN   
            1973             NaN             NaN  ...             NaN   
            1974             NaN             NaN  ...             NaN   
            1975             NaN             NaN  ...             NaN   
            1976             NaN             NaN  ...             NaN   
            1977             NaN             NaN  ...             NaN   
            1978             NaN             NaN  ...             NaN   
            1979             NaN             NaN  ...             NaN   
            1980             NaN             NaN  ...             NaN   
            1981             NaN             NaN  ...             NaN   

Indicator Code    SH.STA.FGMS.ZS  
Country     Year                  
Afghanistan 1972             NaN  
            1973             NaN  
            1974             NaN  
            1975             NaN  
            1976             NaN  
            1977             NaN  
            1978             NaN  
            1979             NaN  
            1980             NaN  
            1981             NaN  

[10 rows x 1594 columns]

#### Save to a pickle file:

In [10]:
#Write data to a pickle file
wb_data.to_pickle(data_dir + world_bank_file_out)